<a href="https://colab.research.google.com/github/lessw2020/training-detr/blob/master/training_detr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Example of how to setup and train your own custom dataset using DETR
# In this notebook, we'll do fine tuning of the existing head
# For larger datasets (say > 10K images) training from scratch may be preferred and we'll do a different notebook for that



In [ ]:
# Credit to @alcinos and @fmassa for their insights, help and for making DETR :) and @mlk1337 and @raviv for additional input 
# thread on custom training with DETR is here:  
# https://github.com/facebookresearch/detr/issues/9

# Setup PyTorch environment and DETR

In [2]:
!pip install torch torchvision


In [3]:
import torch; 
torch_version = torch.__version__
assert float(torch_version[:3]) >= 1.5   # make sure 1.5 or higher
torch_version


'1.5.1+cu101'

In [4]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'   #for coco apis


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-dnv0a1e2
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-dnv0a1e2
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266985 sha256=7158f21d08dc514913f1d9826fe88775f0abbd744e84d3e6fe2f98067bb2d190
  Stored in directory: /tmp/pip-ephem-wheel-cache-4vwsudsx/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.1
    Uninstalling pycocotools-2.0.1:
      Successfully uninstalled pycocotools-2.0.1


In [5]:
!git clone https://github.com/facebookresearch/detr.git  #install detr 


Cloning into 'detr'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 148 (delta 2), reused 13 (delta 0), pack-reused 127
Receiving objects: 100% (148/148), 12.81 MiB | 5.71 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [4]:
from pathlib import Path as p; p.cwd()  # make sure we know where we are

PosixPath('/content')

In [5]:
!ls 

detr  sample_data


In [6]:
%cd /content/detr  
# move default dir to detr to begin...

/content/detr


In [7]:
!ls   # make sure everything looks good

d2			     engine.py	 models		       test_all.py
datasets		     hubconf.py  __pycache__	       tox.ini
detr-r50-e632da11.pth	     LICENSE	 README.md	       util
detr-r50_ready_to_train.pth  logdirs	 requirements.txt
Dockerfile		     main.py	 run_with_submitit.py


# Create a new model with blank fine tuning section for your dataset


In [8]:
from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
#from torchvision.models import resnet50
import torchvision.transforms as T

In [12]:
# download the latest pre-trained weights for detr with resnet50 backbone:
# detr resnet 50 backbone and head
!wget https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth

# or use resnet 101...we'll use res50 for this colab:
# !wget https://dl.fbaipublicfiles.com/detr/detr-r101-2c7b67e5.pth

# Additional info - skip if you like:

# 1 - over time, you want to check if these weight links above become outdated (i.e. newer weights are available) 
# weight listings are located at:  https://github.com/facebookresearch/detr#model-zoo
  
# 2 - Further note - there are DC5 (dc = dilated convolutions) models for both resnet50 and resnet101.  
# Both are a bit more accurate than their regular counterpart, but increase memory consumption 2x 
# and are hard to train as a result (CUDA out of memory errors)...we will use plain resnet50 here.

--2020-06-29 01:26:44--  https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166618694 (159M) [application/octet-stream]
Saving to: ‘detr-r50-e632da11.pth’

detr-r50-e632da11.p 100%[===================>] 158.90M   899KB/s    in 57s     

2020-06-29 01:27:42 (2.79 MB/s) - ‘detr-r50-e632da11.pth’ saved [166618694/166618694]



In [13]:
# we need to load the weights, clip out the two layers we want to train, and then save it back out:
checkpoint = torch.load("detr-r50-e632da11.pth", map_location='cpu')
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]
torch.save(checkpoint,"detr-r50_ready_to_train.pth")


In [ ]:
# build a model for training and adjust num_classes for your dataset

In [9]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader, DistributedSampler

import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model



In [10]:
# pull the args from main.py (the main detr training script) so we can use here in colab to make our model
from main import get_args_parser 

args_parser = get_args_parser()
args = args_parser.parse_args("")

#add our own args for customization
args_parser.add_argument("--num_classes", type=int, default=20)



_StoreAction(option_strings=['--num_classes'], dest='num_classes', nargs=None, const=None, default=20, type=<class 'int'>, choices=None, help=None, metavar=None)

In [ ]:
# a more permanent fix is to adjust main.py directly (or better make a copy of main.py and adjust ala - mymain.py) and add the following:
# # Model parameters
    parser.add_argument('--num_classes', type=int, default=20,
                        help="Number of classes in your dataset. Overridden by coco and coco_panoptic datasets")


In [11]:
args     #check we have a valid args dict with default params from main.py

Namespace(aux_loss=True, backbone='resnet50', batch_size=2, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path=None, dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=300, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_queries=100, num_workers=2, output_dir='', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)

In [12]:
#customize the model - put only your number of classes in your dataset, period. 
args.num_classes =    6#@param


In [ ]:
# Info note on above - DETR maintains a 'no object' class to fill in where no bounding box is predicted.  
# DETR model adds that to your num_classes, so the final, true shape of the predictions is num_classes +1,  
# you only input the actual classes in your dataset above.

In [13]:
# output dir for logging during training
my_detr_logdir = './logdirs/1'   #@param

# makedir and add to args
output_dir = p(my_detr_logdir)
output_dir.mkdir(exist_ok=True, parents=True)  #make if doesn't exist

args.output_dir = str(output_dir)
args.output_dir



'logdirs/1'

In [ ]:
# we have to make an update to ./models/detr.py to the build function to handle our custom number of classes:
# 1 - open /content/detr/models/detr.py
# 2 - find the build function  (def build(args)):
# 2A - remove this block:
     '''num_classes = 20 if args.dataset_file != 'coco' else 91
    if args.dataset_file == "coco_panoptic":
        num_classes = 250'''
#3 - insert this code:
    try:
        num_classes = args.num_classes
    except AttributeError:
        num_classes = 20  # default to 20 for backwards compat if missing args.num_classes

    # over-ride num_classes for known datasets:
    if args.dataset_file == 'coco':
        num_classes = 91

    if args.dataset_file == 'coco_panoptic':
        num_classes = 250

    print(f"---> num_classes = {num_classes}")
    print(f"for dataset:  {args.dataset_file}")
# ----------
# next line should be :  "device = torch.device(args.device)"

# 4 - save detr.py
# reference - PR is in for the above:  https://github.com/facebookresearch/detr/pull/89/commits/d9d98d953e0331f279da377bc53d121a9e171ab7

In [17]:
%load_ext autoreload
%autoreload 2

In [21]:
# we have to update the args.dataset_file to our own name otherwise it will default to 'coco' and thus '91' classes
args.dataset_file = 'mycoco'  #@param

In [23]:
# we are ready to make a model with our custom num_classes.  
model, criterion, postprocessors = build_model(args)
# model.to(device)


---> num_classes = 6
for dataset:  mycoco


In [20]:
args.num_classes

6

In [26]:
# lets inspect our model and make sure custom classes is as we expect:
# we expect the model to have num_classes+1 b/c we have to account for the 'no object class'
m_classes = model.class_embed.out_features
assert(m_classes == args.num_classes+1), "num classes mismatch"
m_classes


7

In [ ]:
# how big is our model: 
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)


number of params: 41302368


In [ ]:
# Customize your dataset  - must be COCO format.
# I have added custom_dataset.py to the repo as a base for handling the class id remapping from your own coco


In [ ]:
# training - in notebook as exposed or run as shell?

In [ ]:
# visuals - check plots for progress


In [ ]:
# inference - run predictions

